## Import libraries

In [40]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import time
import glob


# useful helper function
from helpers import imshow

## Helper methods: general

In [3]:
def initializeMatcher():
    '''
    @returns: FLANN matcher
    '''
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    return flann

In [4]:
def initializeCamera(w, camera_port):
    '''
    @w: width of the video frame
    @returns: camera object
    '''
    camera = cv2.VideoCapture(camera_port)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
    camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
    return camera

In [2]:
def extractFeaturesFromImage(query_path):
    '''
    @query_path: path of the query image
    @returns: keypoints and descriptors of the query image
    '''
    query_img = cv2.imread(query_path, 0)
    kp_query, des_query = sift.detectAndCompute(query_img, None)  
    return kp_query, des_query, query_img

In [101]:
def getGoodMatches(des_query, des_scene):
    '''
    @des_query: descriptors of a query image
    @des_scene: descriptors of a scene image
    @returns: list of good matches for query and scene images
    '''
    
    matches = flann.knnMatch(des_query, des_scene,k=2)

    # ratio test as per Lowe's paper
    good_matches = []
    
    # Each member of the matches list must be checked whether two neighbours really exist.
    for m_n in matches:
        if len(m_n) != 2:
            continue
        (m,n) = m_n
        if m.distance < 0.7*n.distance:
            good_matches.append(m)
            
    return good_matches

In [160]:
def getGoodMatchesForAllSpines(spines, des_scene):
    '''
    @spine: array containing spine keypoints, descriptors and images
    @descriptors of the scene image (video frame)
    @returns: good matches for each of the book spines
    '''
    good_matches = []
    
    for spine in spines:
        des_query = spine[1]
        good_matches.append(getGoodMatches(des_query,des_scene))
    return good_matches

## Helper methods: importing images

In [169]:
def importSpines(directory = 'spines'):
    '''
    @directory: folder where spine images are located
    @returns: array of all grayscale spine images represented as np.arrays, 
    their keypoints and descriptors
    '''
    directory += '/*.jpg'
    spine_fnames = glob.glob(directory)    
    spines = []
    
    for fname in spine_fnames:
        spines.append(extractFeaturesFromImage(fname))
        
    return spines

In [170]:
def importARWindows(spines, dir_covers = 'covers', dir_info = 'info'):
    '''
    @dir_covers: folder where book covers are located
    @dir_info: folder where book info windows are located
    @returns: array of cover window and info window for each book. 
    '''
    dir_covers += '/*.jpg'
    dir_info += '/*.jpg'
    
    cover_fnames = glob.glob(dir_covers)    
    info_fnames = glob.glob(dir_info)    
    
    if len(cover_fnames) != len(info_fnames):
        print "Number of cover windows and info windows is different"
        return None
    
    ar_windows = []

    for i in range(len(cover_fnames)):
        target_height = spines[i][2].shape[0]  # height of the book spine image       
        
        # Cover window
        cover_fname = cover_fnames[i]
        cover_image = cv2.imread(cover_fname)
        # Fit image to book spine
        scaling_factor_cover = 1.0*target_height/cover_image.shape[0]
        cover_image = cv2.resize(cover_image, None, fx = scaling_factor_cover, fy = scaling_factor_cover, interpolation = cv2.INTER_AREA)
        cover_image = cv2.flip(cover_image, 1)

        
        # Book info window
        info_fname = info_fnames[i]
        info_image = cv2.imread(info_fname)
        # Fit image to book spine
        scaling_factor_info = 1.0*target_height/info_image.shape[0]
        info_image = cv2.resize(info_image, None, fx = scaling_factor_info, fy = scaling_factor_info, interpolation = cv2.INTER_AREA)
        info_image = cv2.flip(info_image, 1)
        
        ar_windows.append([cover_image, info_image])
    return ar_windows


## Helper methods: book spine selection

In [161]:
def isSelectedInThisFrame(good_matches, query_img, frame, kp_query, kp_scene):
    '''
    @good_matches: set of good matches
    @query_img: query image
    @frame: video frame image
    @kp_query: keypoints of the query image
    @kp_scene: keypoints of the scene image
    @returns: True if the book was selected in this frame, and False otherwise
    
    ASSUMPTION: keypoints on the book spine are distributed uniformly across the height
    '''
    if len(good_matches) > 15:
        
        # Source points and destnation points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Compute Homography
        M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

        rows,cols = query_img.shape[:2]
        dst = cv2.warpPerspective(frame, M, (cols, rows));
        
        # Theshold matched points that are below the book spine.
        # If less than 1/5 of all good matches i from the lower part of the book spine,
        # register selection of the book spine.
        threshold_height = int(query_img.shape[0] * 0.7)        
        num_matches_in_lower_part = 0
        for point in src_pts:
            if point[0][1] >= threshold_height:
                   num_matches_in_lower_part += 1
        
        book_selected = False
        threshold = int(len(good_matches) / 10)
        print "BOOKSPINE is found"
        print "   ", threshold, num_matches_in_lower_part        
        selected_status = True if num_matches_in_lower_part <= threshold else False
        return selected_status
    
    return False

In [162]:
def checkSelected(good_matches, spines, frame, kp_scene):
    '''
    @good_matches: array of good matches for each book spine
    @spines: keypoints, descriptors and images for all spines
    @frame: current frame
    @kp_scene: keypoints of the current scene (video frame)
    @returns: book that was selected in the current scene, and -1 if no book
    was selected
    '''
    book_selected_in_this_frame = -1
    for i in range(len(spines)):
        query_img = spines[i][2]
        kp_query = spines[i][0]
        if isSelectedInThisFrame(good_matches[i], query_img, frame, kp_query, kp_scene):
            book_selected_in_this_frame = i
            break
    return book_selected_in_this_frame

In [163]:
def updateSelectedStatus(book_selected_in_this_frame, book_selected_in_prev_frame, num_frames_selected):
    '''
    @book_selected_in_this_frame: book selected in the current frame
    @book_selected_in_prev_frame: book selected in the previous frame
    @num_frames_selected: number of consecutive frames that the book was selected
    @returns augmented_book_id: ID of the augmented book, and -1 if no book is 
    selected to be augmented
    @returns num_frames_selected: updated number of consecutive frames that the book 
    was selected
    '''
    
    print "book_selected_in_this_frame", book_selected_in_this_frame
    print "book_selected_in_prev_frame", book_selected_in_prev_frame
    print "NUM FRAMES SELECTED", num_frames_selected
    if book_selected_in_this_frame == -1:
        num_frames_selected = 0
    elif book_selected_in_this_frame == book_selected_in_prev_frame:
        num_frames_selected += 1
    else:
        num_frames_selected = 1

    if num_frames_selected >= 10:
        num_frames_selected = 0
        augmented_book_id = book_selected_in_this_frame
    else:    
        augmented_book_id = -1
    
    return augmented_book_id, num_frames_selected


## Helper methods: augmenting a book spine

In [164]:
def augmentSpine(good_matches, kp_scene, kp_query, frame, window_id, arWindowsSpine):
    '''
    @good_matches: set of good matches for this spine
    @kp_scene: keypoints of the scene image (video frame)
    @kp_query: keypoints of the query image (spine)
    @frame: current video frame
    @window_id: id of the spine's window with which the spine will be augmented
    @arWindowsSpine: libary of augmenting windows foe this spine
    @returns: image of the augmented video frame
    
    '''
    if len(good_matches) > 15:
        # Image of the AR window with which the spine will be augmented
        AR_image = arWindowsSpine[window_id]

        # Source (spine) points and destnation (frame) points
        src_pts = np.float32([kp_query[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_scene[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # Compute Homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        rows,cols = frame.shape[:2]
        dst = cv2.warpPerspective(AR_image, M, (cols, rows));
        
        frame_augmented = frame.copy()
        frame_augmented[np.nonzero(dst)] = dst[np.nonzero(dst)]
        return frame_augmented
    return frame

## Helper methods: input gestures

In [258]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [359]:
def getOpticalFlowDirection(old_gray, frame_gray):
    '''
    @old_gray: previous gray frame
    @frame_gray: new gray frame
    @returns gray_out: a gray image with the displacement vectors added
    @returns: direction of predominant motion
    '''
    gray_out = None
    
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    p1, status, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
   
    
    # Select good points
    good_new = p1[status==1]
    good_old = p0[status==1]
    print "Number of new good matches", len(good_new)
    print "Number of old good matches", len(good_old)
    
    directions = ["RIGHT", "UP", "LEFT", "DOWN"]
    angles = np.zeros(4)
    
    gray_out = frame_gray.copy()
    # Draw Optical Flow
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        
        gray_out = cv2.line(gray_out, (a,b),(c,d), [0, 0, 255], 2)
        gray_out = cv2.circle(gray_out,(a,b),5,[0, 0, 255],-1)

        # Calculate strongest direction of the flow
        dx = a - c
        dy = b - d
        
        if abs(dx) > abs(dy):            
            mag = dx
            if abs(mag) > 30:
                if mag > 0:
                    angles[2] += 1
                else:
                    angles[0] += 1
        else:          
            mag = dy
            if abs(mag) > 30:
                if mag > 0:
                    angles[3] += 1
                else:
                    angles[1] += 1      

    if np.sum(angles) > 0:
        print np.max(angles), directions[np.argmax(angles)]

    return gray_out, np.max(angles)

# Main system code

In [362]:
from IPython.display import clear_output

# SIFT keypoint extractor
sift = cv2.xfeatures2d.SIFT_create()

spines = importSpines()
arWindows = importARWindows(spines)

# Initialize FLANN matcher
flann = initializeMatcher()

# Initialize camera
camera = initializeCamera(640, 1)

# Initialize parameters: Book Selection
book_selected_in_prev_frame = -1 # book selected in previous frame
num_frames_selected = 0 # number of consecutive frames in which the current book spine was selected
augmented_book_id = -1 # Book for which AR menu is displayed

# Initialize parameters: Augmentation
aug_window_id = 1 # Initialize id of the augmenting window

# Initialize parameters: Optical Flow
old_frame = None


while True:
    clear_output(True)

    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # keypoints and descriptors for video frame 
    kp_scene, des_scene =  sift.detectAndCompute(frame_gray,None)
    
    if augmented_book_id == -1:
        
        # Find good matches for each book spine
        good_matches = getGoodMatchesForAllSpines(spines, des_scene)
  
        # Check if any of the bookspines has been selected
        # If selected 10 times in a row, change this book's status to Selected
        book_selected_in_this_frame = checkSelected(good_matches, spines, frame, kp_scene)
        augmented_book_id, num_frames_selected = updateSelectedStatus(book_selected_in_this_frame, book_selected_in_prev_frame, num_frames_selected)
        
        book_selected_in_prev_frame = book_selected_in_this_frame
    else:
        print "SELECTED", augmented_book_id
        
        spine = spines[augmented_book_id]
        kp_query = spine[0]
        arWindowsSpine = arWindows[augmented_book_id]
        
        good_matches_aug = getGoodMatchesForAllSpines([spine], des_scene)[0]
        augmented_frame = augmentSpine(good_matches_aug, kp_scene, kp_query, frame, aug_window_id, arWindowsSpine)
        frame = augmented_frame
        
        
        #-------------- Optical Flow ------------------------
        gray_out, flow_direction = getOpticalFlowDirection(old_gray, frame_gray)
        if gray_out is not None:
            gray_out = cv2.flip(gray_out, 1)
            cv2.imshow("GRAY", gray_out)
       
    if cv2.waitKey(5) == 32:
        augmented_book_id = -1
    elif cv2.waitKey(5) == 27:
        break  
    
    frame = cv2.flip(frame, 1)
    cv2.imshow("Frame", frame)
    
    # Update parameters for optical flow:
    old_gray = frame_gray.copy()
        
cv2.destroyAllWindows()
camera.release()


SELECTED 0


error: /tmp/opencv-20180110-77632-pyvb0m/opencv-3.4.0/modules/flann/src/miniflann.cpp:315: error: (-210) type=0
 in function buildIndex_


# Optical flow on coarse grayscale images

In [372]:
# parameters for farneback optical flow
fb_params = dict( pyr_scale = 0.5, 
                  levels = 3, 
                  winsize = 5, 
                  iterations = 3, 
                  poly_n = 5,
                  poly_sigma = 1.2, 
                  flags = 0 )

In [369]:
def getOrigFrameCoord(x):
    '''
    @x: coordinate in a scaled coordinate system
    @returns: coordinate in the original coordinate system
    '''
    global scaling_factor_OF
    scaling_factor_OF = 0.2
    return int(x / scaling_factor_OF)

In [383]:
def getSwipeDirection(prev_gray, next_gray, frame):
    '''
    @old_gray: previous gray frame
    @frame_gray: new gray frame
    
    @returns img_out: An image with the optical flow - based controller added
    @returns swipeDirection: direction of the swipe
    '''
    
    # Calculate the dense optical flow on reduced gray images
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, **fb_params)

    stride = int(next_gray.shape[1] * 0.39)
    
    img_out = frame.copy()

    gestures = ["RIGHT","LEFT"]
    activation_threshold = 7

    # Plot the controller and find its displacement:
    x_pos_start = next_gray.shape[1] / 2 - stride
    y_pos_start = next_gray.shape[0] / 2 - stride/2
    scale=2

    for j in range(2):
        for i in range(3):    
            x = x_pos_start + i * stride
            y = y_pos_start + j * stride
            
            dx, dy = flow[...,0][y, x], flow[...,1][y, x]
            
            if i == 1 and j == 1:
                # Draw the controller
                cv2.line(img_out, (getOrigFrameCoord(x), getOrigFrameCoord(y)), (getOrigFrameCoord(x + int(dx * scale)), getOrigFrameCoord(y + int(dy * scale))), (0, 0, 255), 8)
                cv2.circle(img_out, (getOrigFrameCoord(x + int(dx * scale)), getOrigFrameCoord(y + int(dy * scale))), 10, (0, 0, 255), 8) 
                
                # Draw boundary lines for activation threshold
                cv2.line(img_out, (getOrigFrameCoord(x-activation_threshold*scale), getOrigFrameCoord(y-5)), (getOrigFrameCoord(x-activation_threshold*scale), getOrigFrameCoord(y+5)), (0, 0, 0), 2)
                cv2.line(img_out, (getOrigFrameCoord(x+activation_threshold*scale), getOrigFrameCoord(y-5)), (getOrigFrameCoord(x+activation_threshold*scale), getOrigFrameCoord(y+5)), (0, 0, 0), 2)
                
                swipeDirection = -1
                if abs(dx) > activation_threshold:
                    swipeDirection = 0 if dx >= 0 else 1
                    print gestures[swipeDirection]
    return img_out, swipeDirection

In [388]:
from IPython.display import clear_output

# SIFT keypoint extractor
sift = cv2.xfeatures2d.SIFT_create()

spines = importSpines()
arWindows = importARWindows(spines)

# Initialize FLANN matcher
flann = initializeMatcher()

# Initialize camera
camera = initializeCamera(640, 1)

# Initialize parameters: Book Selection
book_selected_in_prev_frame = -1 # book selected in previous frame
num_frames_selected = 0 # number of consecutive frames in which the current book spine was selected
augmented_book_id = -1 # Book for which AR menu is displayed

# Initialize parameters: Augmentation
aug_window_id = 1 # Initialize id of the augmenting window

# Initialize parameters: Optical Flow
global scaling_factor_OF
scaling_factor_OF = 0.2

while True:
    clear_output(True)

    # Get frame at flip it
    ret, frame = camera.read()
    frame = cv2.flip(frame, 1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize grame_gray for coarse-to-fine optical flow
    new_gray_OF = cv2.resize(frame_gray.copy(), None, fx = scaling_factor_OF, fy = scaling_factor_OF, interpolation = cv2.INTER_AREA)

    # keypoints and descriptors for video frame 
    kp_scene, des_scene =  sift.detectAndCompute(frame_gray,None)
    
    if augmented_book_id == -1:
        
        # Find good matches for each book spine
        good_matches = getGoodMatchesForAllSpines(spines, des_scene)
  
        # Check if any of the bookspines has been selected
        # If selected 10 times in a row, change this book's status to Selected
        book_selected_in_this_frame = checkSelected(good_matches, spines, frame, kp_scene)
        augmented_book_id, num_frames_selected = updateSelectedStatus(book_selected_in_this_frame, book_selected_in_prev_frame, num_frames_selected)
        
        book_selected_in_prev_frame = book_selected_in_this_frame
    else:
        print "SELECTED", augmented_book_id
        
        spine = spines[augmented_book_id]
        kp_query = spine[0]
        arWindowsSpine = arWindows[augmented_book_id]
        
        good_matches_aug = getGoodMatchesForAllSpines([spine], des_scene)[0]
        augmented_frame = augmentSpine(good_matches_aug, kp_scene, kp_query, frame, aug_window_id, arWindowsSpine)
        frame = augmented_frame
        
        
#         #-------------- Optical Flow ------------------------
#         frameOF, swipeDirection = getSwipeDirection(old_gray_OF, new_gray_OF, frame)
#         frameOF = cv2.flip(frameOF, 1)
#         cv2.imshow("Optical Flow", frameOF)
       
    if cv2.waitKey(5) == 32:
        augmented_book_id = -1
    elif cv2.waitKey(5) == 27:
        break  
    
    frame = cv2.flip(frame, 1)
    cv2.imshow("Frame", frame)
    
    # Update old_gray_OF for coarse-to-fine optical flow
    old_gray_OF = new_gray_OF.copy()
        
cv2.destroyAllWindows()
camera.release()

SELECTED 1
